In [1]:
!pip install numpy==1.23.0  # Or try other older versions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 10.6 MB/s eta 0:00:0000:010:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [1262 lines of output]
      setup.py:71: RuntimeWarning: NumPy 1.23.0 may not yet support Python 3.11.
        warnings.warn(
      Running from numpy source directory.
      setup.py:86: DeprecationWarning:
      
        `numpy.distutils` is deprecated since NumPy 1.23.0, as a result
        of the deprecation of `distutils` itself. It will be removed for
        Python >= 3.12. For older Python versions it will remain present.
        It is recommended to use `setuptools < 60.0` for those Python versions.
        For more details, see:
          https://numpy.org/devdocs/reference/distutils_status_migration.html
      
      
 

In [2]:
# !pip install essentia --force-reinstall --no-cache-dir
import os
# Force the array API to be NumPy within the global scope
import numpy as np
os.environ['_ARRAY_API'] = 'numpy' # Sets the array API for Essentia

import essentia #Now import Essentia after setting the environment variable
import essentia.standard as es
import concurrent.futures  # For parallel processing
import time

#If CUDA is supported
try:
    from essentia import cuda
    use_cuda = True
except:
    print("CUDA not supported in your essenta build.")
    use_cuda = False #Make it not run on GPU if not supported

def extract_audio_features(audio_file, use_cuda=False):
    """
    Extracts audio features from a single audio file using Essentia.

    Args:
        audio_file (str): Path to the audio file (.wav).
        use_cuda (bool): Whether to use GPU acceleration (if available).

    Returns:
        dict: A dictionary containing extracted audio features, or None if an error occurs.
    """
    try:
        if use_cuda:
            # Create loader and connect with the GPU using essentia.cuda
            loader = es.MonoLoader(filename=audio_file, sampleRate=44100) #Example sample rate of 44100
            loader_cuda = cuda.MonoLoader(loader) #cuda accelerated version of the monoloader
            audio = loader_cuda() #Load

        else:
            audio = es.MonoLoader(filename=audio_file, sampleRate=44100)() #Example sample rate of 44100

        # 1. Extract Low-Level Features
        lowlevel = es.LowLevelDescriptors() #Returns a named tuple of all the values, including the RMS
        if use_cuda:
            lowlevel_cuda = cuda.LowLevelDescriptors(lowlevel)
            features_lowlevel = lowlevel_cuda(audio)
        else:
            features_lowlevel = lowlevel(audio)
        #These low-level descriptiors include all the descriptors
        #RMS - features_lowlevel[1]
        #Spectral Centroid - features_lowlevel[7]
        #Zero-crossing rate - features_lowlevel[0]


        # 2. More Complex Descriptors - Extract MFCCs
        windowing = es.Windowing(type='hann') #Windowing for each frame
        spectrum = es.Spectrum()
        mfcc = es.MFCC()

        mfccs_all_frames = []
        #Now run through audio frames for time information
        for frame in es.FrameGenerator(audio, frameSize = 2048, hopSize = 512): #Example Frame Size, adjust accordingly
            # Apply Windowing
            frame_w = windowing(frame)
            #Create the spectrum for the frame with CUDA implementation
            X = spectrum(frame_w) # Get the frequency domain
            #Extract the values of the MFFCs
            mfcc_bands, mfcc_coeffs = mfcc(X)
            mfccs_all_frames.append(mfcc_coeffs)

        # Convert MFCCs to numpy array - Shape is (num_frames, num_mfccs)
        mfccs_all_frames = np.array(mfccs_all_frames) #Numpy for good analysis

        extracted_features = {
            "rms": features_lowlevel[1], #Second element in the named tuple
            "spectral_centroid": features_lowlevel[7], #8th element in the named tuple
            "zero_crossing_rate":features_lowlevel[0],# First element in the name tuple
            "mean_mfccs": np.mean(mfccs_all_frames, axis=0), # Averaged MFCCs
            "std_mfccs": np.std(mfccs_all_frames, axis=0), # MFCC standard deviation
            # Add more features here as needed
        }

        return extracted_features

    except Exception as e:
        print(f"Error processing audio file {audio_file}: {e}")
        return None

def process_audio_file(audio_file, output_folder, use_cuda):
    """Processes the audio file, and generates the folder"""
    # Get the base name
    file_name = os.path.basename(audio_file)
    file_name_without_ext, file_ext = os.path.splitext(file_name)
    output_txt_file = os.path.join(output_folder, f"{file_name_without_ext}_features.txt")
    print(f"Processing audio: {audio_file}")

    try:
        features = extract_audio_features(audio_file, use_cuda)

        if features:
            with open(output_txt_file, "w") as f:
                for key, value in features.items():
                    f.write(f"{key}: {value}\n")
            print(f"Features saved to {output_txt_file}")
        else:
            print("No features extracted.")

    except Exception as e:
        print(f"Error in process_audio_file: {e}")

def traverse_directory(root_directory, output_root_folder, max_workers=4, use_cuda=False):
    """
    Traverses a directory, processes all audio files, and extracts information for each file.
    Args:
        root_directory (string): the starting directory
        output_root_folder (string) : the directory to be put to
        max_workers (int) : the maximum number of workers
        use_cuda (bool) : whether the library should use CUDA to accelerate operations
    """
    audio_files = [] # The audio files to be worked on
    for foldername, subfolders, filenames in os.walk(root_directory):
        for filename in filenames:
            if filename.lower().endswith(('.wav')):
                input_file_path = os.path.join(foldername, filename)
                relative_path = os.path.relpath(foldername, root_directory)
                output_folder = os.path.join(output_root_folder, relative_path)
                os.makedirs(output_folder, exist_ok=True)
                audio_files.append((input_file_path, output_folder))  # Store path and folder

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(lambda item: process_audio_file(item[0], item[1], use_cuda), audio_files)  # Process in parallel

# Example usage:

if __name__ == '__main__':
    input_directory = "/content/drive/MyDrive/Datasets/preprocess"  # Replace with your input directory
    output_directory = "/content/drive/MyDrive/Datasets/AudioFeatures/"  # Replace with your output directory

    start_time = time.time()
    traverse_directory(input_directory, output_directory, max_workers = 4, use_cuda=use_cuda) # Use GPU, if it works
    end_time = time.time()
    print("Time is", end_time-start_time, "seconds") #See total time to make adjustments


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/kushagraagarwal/Documents/Capstone_94/cap/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/kushagraagarwal/Documents/Capstone_94/cap/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/kushagraagarwal/Documents/Capstone_94/cap/lib/python3.11/site-packages/ipykernel/

AttributeError: _ARRAY_API not found

In [1]:
print(type("Hello"))

<class 'str'>


In [5]:
import time
start_time = time.time()
print(type(start_time))

<class 'float'>
